## Import modules

In [ ]:
import os
import sys
import math
from collections import OrderedDict
import copy
import random
import glob
from easydict import EasyDict

import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torch.nn.init as init
import torch.utils.data as data
import torch.utils.data.dataset as dataset
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score
import torchvision.utils as v_utils
import matplotlib.pyplot as plt
import time
from models.MNAD.model.utils import CustomDataLoader
from models.MNAD.model.final_future_prediction_with_memory_spatial_sumonly_weight_ranking_top1 import *
from models.MNAD.model.Reconstruction import *
from models.MNAD.utils import *
import urls

## 1 Set configs and environment

In [ ]:
cfg = EasyDict(
    # dataset
    video_folder='/home/yuanyu/projects/data/DaYanTa_2/8_C51/frames',
    label_file_path='/home/yuanyu/projects/dyt_VAD/label.csv',
    transform=transforms.Compose([transforms.ToTensor()]),
    resized_height=256,
    resized_width=256,
    time_step=4,
    num_pred=1,

    # model
    method='pred',
    t_length=5,
    fdim=512,
    mdim=512,
    msize=10,

    # test
    alpha=1,
    th=0.01,
    num_workers=1,
    model_path='./models/MNAD/exp/Xian/pred/log/model.pth',
    m_items_path='./models/MNAD/exp/Xian/pred/log/keys.pt',
    loss_func_mse=nn.MSELoss(reduction='none'),
    gpus='1,2,3',
    batch_size=1
)

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = cfg.gpus
# make sure to use cudnn for computational performance
torch.backends.cudnn.enabled = True

## 2 Load the dataset and model

In [ ]:
# init dataloader
test_dataset = CustomDataLoader(cfg.video_folder, transforms.Compose([
    transforms.ToTensor(),
]), label_file_path=cfg.label_file_path,
    resize_height=cfg.resized_height, resize_width=cfg.resized_width, time_step=cfg.t_length-1, train=False)

test_loader = data.DataLoader(test_dataset, batch_size=cfg.batch_size,
                             shuffle=False, num_workers=cfg.num_workers, drop_last=False)

In [ ]:
# init model
model = torch.load(cfg.model_path)
model.cuda()
model.eval()
m_items = torch.load(cfg.m_items_path)


In [ ]:
# log and test data_loader
print(len(test_loader))
for data in test_loader:
    print(type(data), data.shape)
    break

In [ ]:
# log and test model
for imgs in test_loader:
    imgs = imgs.cuda()
    outputs, feas, updated_feas, m_items, softmax_score_query, softmax_score_memory, _, _, _, compactness_loss = model.forward(
        imgs[:, 0:3*4], m_items, False
    )
    mse_imgs = cfg.loss_func_mse((outputs[0]+1)/2, (imgs[0, 3*4:]+1)/2)
    mse_loss = torch.mean(mse_imgs).item()
    mse_feas = compactness_loss.item()
    
    print(mse_loss)
    break

## 3 Run the model on dataloader, and save results

In [ ]:
# Init result savers
results = dict()

video_name_list = list()

mse_imgs_list = list()
mse_loss_list = list()
psnr_list = list()

In [ ]:
for imgs in test_loader:
    imgs = imgs.cuda()
    outputs, feas, updated_feas, m_items, softmax_score_query, softmax_score_memory, _, _, _, compactness_loss = model.forward(
        imgs[:, 0:3*4], m_items, False
    )
    mse_imgs = cfg.loss_func_mse((outputs[0]+1)/2, (imgs[0, 3*4:]+1)/2)
    mse_loss = torch.mean(mse_imgs).item()
    mse_feas = compactness_loss.item()
    psnr_ = psnr(mse_loss)

    # print(mse_loss)
    mse_imgs_list.append(mse_imgs)
    mse_loss_list.append(mse_loss)
    psnr_list.append(psnr_)